**LOADING LIBRARIES**

In [1]:
import pandas as pd
import numpy as np
import os
import torch
from torch import optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
device

device(type='cpu')

**LOADING DATA**

In [3]:
for dirname, _, filenames in os.walk('/kaggle/input/'):
    if len(filenames) != 0:
        if filenames[0] != "sample_submission.csv":
            avg = np.array([os.path.getsize(os.path.join(dirname, filename)) for filename in filenames]).mean()
            # os.path.getsize returns the size of the dictionary passed in bytes
            print(dirname, len(os.listdir(dirname)))
            print("Size: {:.3f} KB".format(avg/1024)) # 1024 is 1KB

/kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla/valid 676
Size: 22.240 KB
/kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla/test 844
Size: 17.657 KB
/kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla/train 5709
Size: 26.440 KB
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/random/valid 20
Size: 12802.166 KB
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/random/test 17
Size: 270.353 KB
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/random/train 207
Size: 11594.272 KB
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/default/valid 20
Size: 11852.173 KB
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/default/test 17
Size: 262.961 KB
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/default/train 198
Size: 10526.417 KB
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/xla/random/valid 7
Size: 6434.424 KB
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/xla/rand

In [4]:
tile = np.load('/kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla/train/retinanet.4x4.fp32_-431a58cc30e72ec6.npz')
tile.files

['node_feat',
 'node_opcode',
 'edge_index',
 'config_feat',
 'config_runtime',
 'config_runtime_normalizers']

In [5]:
basic_structure = tile.files
for dirname, _, filenames in os.walk('/kaggle/input'):
    flag = False
    for filename in filenames:
        if filename != "sample_submission.csv" and filename[-4:] == '.npz':
            
            if np.load(os.path.join(dirname, filename)).files != basic_structure and not flag:
                print(dirname)
                print(np.load(os.path.join(dirname, filename)).files)
                flag = True

/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/random/valid
['edge_index', 'node_feat', 'node_opcode', 'node_config_feat', 'node_config_ids', 'node_splits', 'config_runtime']
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/random/test
['edge_index', 'node_feat', 'node_opcode', 'node_config_feat', 'node_config_ids', 'node_splits', 'config_runtime']
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/random/train
['edge_index', 'node_feat', 'node_opcode', 'node_config_feat', 'node_config_ids', 'node_splits', 'config_runtime']
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/default/valid
['edge_index', 'node_feat', 'node_opcode', 'node_config_feat', 'node_config_ids', 'node_splits', 'config_runtime']
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/default/test
['edge_index', 'node_feat', 'node_opcode', 'node_config_feat', 'node_config_ids', 'node_splits', 'config_runtime']
/kaggle/input/predict-ai-model-runtime/npz_all/npz

In [6]:
def load_data(directory):
    splits = ['train', 'valid', 'test']
    dfs = dict()
    for split in splits:
        path = os.path.join(directory, split)
        files = os.listdir(path)
        list_df = []
        
        for file in files:
            list_df.append(dict(np.load(os.path.join(path, file))))
        dfs[split] = pd.DataFrame.from_dict(list_df)
    return dfs

**DATA PREPROCESSING**

In [7]:
def make_datum(datum, i):
    configs = datum.config_feat[i].shape[0]
    graph = pd.DataFrame(columns=['node_feat', 'node_opcode', 'edge_index', 'config_feat', 'config_runtime', 'config_runtime_normalizers'])
    for config in range(0, configs):
        Sample = pd.Series()
        Sample['node_feat'] = datum.node_feat[i]
        Sample['node_opcode'] = datum.node_opcode[i]
        Sample['edge_index'] = datum.edge_index[i]
        Sample['config_feat'] = datum.config_feat[i][config]
        Sample['config_runtime'] = datum.config_runtime[i][config]
        Sample['config_runtime_normalizers'] = datum.config_runtime_normalizers[i][config]
        Sample['topo_order'] = datum.topo_order.values[0]
        graph = pd.concat([graph, Sample.to_frame().T], ignore_index=True)
    return graph


In [8]:
def make_data(data, b=0):
    df = pd.DataFrame(columns = data.columns)
    for i in range(len(data)):
        datum = data.iloc[i].to_frame().T
        graph = make_datum(datum, i+b)
        df = pd.concat([df, graph], axis=0, ignore_index=True)
    df['avg_runtime'] = df['config_runtime'] / (df['config_runtime_normalizers'] + 1e-5)
    df.drop(['config_runtime', 'config_runtime_normalizers'], axis=1, inplace=True)
    return df

In [9]:
class dataset(Dataset):
    def __init__(self, data):
        self.feats = data[data.columns[:-1]]
        self.labels = data[data.columns[-1]]
        self.transform = transforms.Compose([transforms.ToTensor()])
    
    def __len__(self):
        return len(self.feats)
    
    def __getitem__(self, idx):
        # early join of config features
        op_code = torch.tensor(self.feats.node_opcode.iloc[idx])
        node_f = torch.tensor(self.feats.node_feat.iloc[idx])
        config_f = torch.tensor(self.feats.config_feat.iloc[idx])
        adj_mat = torch.tensor(self.feats.edge_index.iloc[idx])
        topo_o = torch.tensor(self.feats.topo_order.iloc[idx])
        config_f_broadcasted = config_f.unsqueeze(0).expand(node_f.size(0), -1)
        node_config_f = torch.cat((node_f, config_f_broadcasted), axis=1)
        # padding for compatibility with the model GNN
        to_add_rows = 500 - node_config_f.size(0)
        node_config_row_zeros = torch.zeros((to_add_rows, node_config_f.size(1)), dtype=node_config_f.dtype)
        node_config_f = torch.cat((node_config_f, node_config_row_zeros), dim=0)
        op_top_zeros = torch.zeros(to_add_rows, dtype=op_code.dtype)
        op_code = torch.cat((op_code, op_top_zeros))
        topo_o = torch.cat((topo_o, op_top_zeros))
        adj_row_zeros = torch.zeros((to_add_rows, adj_mat.size(1)), dtype=adj_mat.dtype)
        adj_mat = torch.cat((adj_mat, adj_row_zeros), dim=0)
        adj_col_zeros = torch.zeros((adj_mat.size(0), to_add_rows), dtype=adj_mat.dtype)
        adj_mat = torch.cat((adj_mat, adj_col_zeros), dim=1)
        # padding done (nodes == 100)
        inputs = [op_code, node_config_f, adj_mat, topo_o]
        return inputs, torch.tensor(self.labels.iloc[idx])

In [10]:
test = False
if(test):
    custom_dataset = dataset(made_tilex_train)
    loader = DataLoader(custom_dataset, batch_size=2, shuffle=True)
    for batch in loader:
        break

**BUILDING MODEL GNN**

In [11]:
test = False
if(test):
    gnn = GNN(174)
    batch_feats, batch_true = batch
    batch_preds = gnn(batch_feats)
    print(batch_preds)

In [12]:
def adj_mat(edges, n):
    # edges are 0-indexed
    adj_matrix = np.zeros((n, n))
    for u, v in edges:
        adj_matrix[u][v] = 1
        adj_matrix[v][u] = 1
    return adj_matrix

In [13]:
adj_matrix = adj_mat([[2, 3], [3, 1]], 4)
print(type(adj_matrix))
adj_matrix

<class 'numpy.ndarray'>


array([[0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 1., 0.]])

In [14]:
def topo_sort(edges):
    indeg = {}
    for edge in edges:
        if(edge[0] not in indeg):
            indeg[edge[0]] = 0
        if(edge[1] not in indeg):
            indeg[edge[1]] = 0
        indeg[edge[1]] += 1
    
    queue = []
    for node in indeg:
        if(indeg[node] == 0):
            queue.append(node)
    topo_order = []
    while(queue):
        node = queue.pop()
        topo_order.append(node)
        for neighbour in [edge[1] for edge in edges if edge[0] == node]:
            indeg[neighbour] -= 1
            if(indeg[neighbour] == 0): 
                queue.append(neighbour)
    # let's reorder the topological sort to be compatible with topo-order aware downsampling
    topo_embd = []
    for i, node in enumerate(topo_order):
        topo_embd.append((node, i))
    topo_embd = sorted(topo_embd)
    topo_order = []
    for node, i in topo_embd:
        topo_order.append(i)
    return topo_order

In [15]:
edges = [[2, 3], [3, 1], [1, 4], [3, 4]]
topo_sort(edges)

[2, 0, 1, 3]

In [16]:
def kipf_norm(adj_mats):
    norm_mats = []
    for adj_mat in adj_mats:
        max_degree = torch.max(torch.sum(adj_mat, dim=1))
        # add inv-deg term if convergence is slow
        norm_mat = adj_mat / max_degree
        norm_mats.append(norm_mat)
    return torch.stack(norm_mats, axis=0)

**TILE CONFIGURATION**

In [17]:
tile_xla = load_data('/kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla/')

In [18]:
tilex_train = tile_xla['train']
tilex_valid = tile_xla['valid']
tilex_test = tile_xla['test']


In [19]:
print(tilex_train.shape)
# convert all the edge_index to adjacency matrix
# also add topo_order of edge_index
tilex_train.head()

(5709, 6)


,node_feat,node_opcode,edge_index,config_feat,config_runtime,config_runtime_normalizers
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 63, 2, 63, 11, 63, 11, 63, 24,...","[[2, 0], [2, 1], [5, 3], [5, 4], [7, 6], [9, 8...","[[16.0, 16.0, 1.0, 2.0, 0.0, 0.0, 35.0, 512.0,...","[238408, 5052251, 1873104, 2452158, 1872430, 1...","[238408, 238408, 238408, 238408, 238408, 23840..."
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 25, 63, 11, 63, 41, 63, 41, 26...","[[2, 0], [2, 1], [4, 3], [6, 5], [8, 6], [8, 7...","[[3.0, 3.0, 32.0, 2.0, 0.0, 0.0, 40.0, 576.0, ...","[1003878, 42948537, 6658438, 80260521, 2511620...","[1003878, 1003878, 1003878, 1003878, 1003878, ..."
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 11, 63, 11, 63, 25, 63, 13, 95...","[[2, 0], [2, 1], [4, 3], [6, 5], [8, 7], [10, ...","[[1.0, 3.0, 103.0, 2.0, 0.0, 0.0, 109.0, 618.0...","[23134, 421195, 421565, 88111, 155930, 176701,...","[23134, 23134, 23134, 23134, 23134, 23134, 231..."
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 63, 2, 63, 2, 25, 63, 13, 95, ...","[[2, 0], [2, 1], [5, 3], [5, 4], [7, 5], [7, 6...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 32.0...","[25371, 26002, 39148, 25318, 58177, 31551, 253...","[25371, 25371, 25371, 25318, 25318, 25318, 253..."
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 25, 63, 13, 63, 25, 63, 13, 95...","[[2, 0], [2, 1], [4, 3], [6, 5], [8, 7], [10, ...","[[1.0, 1.0, 64.0, 2.0, 0.0, 0.0, 68.0, 128.0, ...","[100762, 553632, 986995, 590880, 793928, 16255...","[100762, 100762, 100762, 100762, 100762, 10076..."


In [20]:
# prepare train data 1
test = False
if test:
    tile_train = tilex_train.head(len(tilex_train)//2).copy(deep=True)
    tile_train['topo_order'] = tile_train['edge_index'].map(lambda e: topo_sort(e))
    tile_train['edge_index'] = tile_train['edge_index'].map(lambda e: adj_mat(e, max([max(edge) for edge in e])+1))
    tile_train = make_data(tile_train)
    tile_train.to_pickle("/kaggle/working/tiler_train1.pkl")

In [21]:
# prepare train data 2
test = False
if test:
    b = len(tilex_train)//2
    tile_train = tilex_train.tail(len(tilex_train)-b).copy(deep=True)
    tile_train['topo_order'] = tile_train['edge_index'].map(lambda e: topo_sort(e))
    tile_train['edge_index'] = tile_train['edge_index'].map(lambda e: adj_mat(e, max([max(edge) for edge in e])+1))
    tile_train = make_data(tile_train, b)
    tile_train.to_pickle('/kaggle/working/tiler_train2.pkl')

In [22]:
# prepare valid data
test = False
if test:
    tile_val = tilex_valid.copy(deep=True)
    tile_val['topo_order'] = tile_val['edge_index'].map(lambda e: topo_sort(e))
    tile_val['edge_index'] = tile_val['edge_index'].map(lambda e: adj_mat(e, max([max(edge) for edge in e])+1))
    tile_val = make_data(tile_val)
    tile_val.to_pickle('/kaggle/working/tiler_val.pkl')

In [23]:
# test GNN, gConv, data preprocessing steps
toy = tilex_train.head(1).copy(deep=True)
n = toy.node_feat[0].shape[0]
toy['topo_order'] = toy['edge_index'].map(lambda e: topo_sort(e))

In [24]:
# edge_index -> adj_matrix
toy['edge_index'] = toy['edge_index'].map(lambda e: adj_mat(e, n))
toy

,node_feat,node_opcode,edge_index,config_feat,config_runtime,config_runtime_normalizers,topo_order
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 63, 2, 63, 11, 63, 11, 63, 24,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[16.0, 16.0, 1.0, 2.0, 0.0, 0.0, 35.0, 512.0,...","[238408, 5052251, 1873104, 2452158, 1872430, 1...","[238408, 238408, 238408, 238408, 238408, 23840...","[30, 29, 28, 9, 8, 7, 23, 22, 19, 18, 27, 26, ..."


In [25]:
print(toy.edge_index[0].shape)
toy

(31, 31)


,node_feat,node_opcode,edge_index,config_feat,config_runtime,config_runtime_normalizers,topo_order
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 63, 2, 63, 11, 63, 11, 63, 24,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[16.0, 16.0, 1.0, 2.0, 0.0, 0.0, 35.0, 512.0,...","[238408, 5052251, 1873104, 2452158, 1872430, 1...","[238408, 238408, 238408, 238408, 238408, 23840...","[30, 29, 28, 9, 8, 7, 23, 22, 19, 18, 27, 26, ..."


In [26]:
# apply GNN, gConv on toy
import warnings
warnings.filterwarnings('ignore')
made_toy = make_data(toy)
made_toy

,node_feat,node_opcode,edge_index,config_feat,topo_order,avg_runtime
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 63, 2, 63, 11, 63, 11, 63, 24,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[16.0, 16.0, 1.0, 2.0, 0.0, 0.0, 35.0, 512.0, ...","[30, 29, 28, 9, 8, 7, 23, 22, 19, 18, 27, 26, ...",1.0
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 63, 2, 63, 11, 63, 11, 63, 24,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[3.0, 2.0, 1.0, 1.0, 0.0, 0.0, 7.0, 6.0, 1.0, ...","[30, 29, 28, 9, 8, 7, 23, 22, 19, 18, 27, 26, ...",21.191617
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 63, 2, 63, 11, 63, 11, 63, 24,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[1.0, 14.0, 1.0, 1.0, 0.0, 0.0, 17.0, 14.0, 1....","[30, 29, 28, 9, 8, 7, 23, 22, 19, 18, 27, 26, ...",7.856716
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 63, 2, 63, 11, 63, 11, 63, 24,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[14.0, 1.0, 1.0, 1.0, 0.0, 0.0, 17.0, 14.0, 1....","[30, 29, 28, 9, 8, 7, 23, 22, 19, 18, 27, 26, ...",10.285552
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 63, 2, 63, 11, 63, 11, 63, 24,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[12.0, 2.0, 1.0, 1.0, 0.0, 0.0, 16.0, 24.0, 1....","[30, 29, 28, 9, 8, 7, 23, 22, 19, 18, 27, 26, ...",7.853889
...,...,...,...,...,...,...
1532,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 63, 2, 63, 11, 63, 11, 63, 24,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[80.0, 7.0, 1.0, 1.0, 0.0, 0.0, 89.0, 560.0, 1...","[30, 29, 28, 9, 8, 7, 23, 22, 19, 18, 27, 26, ...",9.021717
1533,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 63, 2, 63, 11, 63, 11, 63, 24,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[40.0, 10.0, 1.0, 2.0, 0.0, 0.0, 53.0, 800.0, ...","[30, 29, 28, 9, 8, 7, 23, 22, 19, 18, 27, 26, ...",1.761068
1534,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 63, 2, 63, 11, 63, 11, 63, 24,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[20.0, 20.0, 1.0, 2.0, 0.0, 0.0, 43.0, 800.0, ...","[30, 29, 28, 9, 8, 7, 23, 22, 19, 18, 27, 26, ...",1.168978
1535,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[63, 63, 2, 63, 63, 2, 63, 11, 63, 11, 63, 24,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[3.0, 80.0, 1.0, 2.0, 0.0, 0.0, 86.0, 480.0, 1...","[30, 29, 28, 9, 8, 7, 23, 22, 19, 18, 27, 26, ...",1.011211


In [27]:
# make data into batchs and test it on gConv and GNN
custom_dataset = dataset(made_toy)
loader = DataLoader(custom_dataset, batch_size=2, shuffle=True)
for batch in loader:
    break
batch

[[tensor([[ 63,  63,   2,  63,  63,   2,  63,  11,  63,  11,  63,  24,  13,  59,
            63,  41,  63,  41,  26,  25,   2,  59,  24,  70,  59,  70, 100,  63,
            63,  63,  41,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             0,   0,   0,   

In [28]:
# use batch to test the model
test = False
if test:
    batch_feat, batch_labels = batch
    model = GNN(174, 2)
    model = model.to(device)
    batch_preds = model(batch_feat)
    batch_preds

In [29]:
avg = np.array([os.path.getsize(os.path.join(dirname, filename)) for filename in filenames]).mean()
# what does this mean, something related to the size of the data?

In [30]:
sample = tilex_train.head(1)
print('no of nodes: (#nodes) in computation graph ', len(sample.node_feat[0]))
print('node_feat: (#features_of_a_node) ', len(sample.node_feat[0][0]))
print('node_opcode: (#operation_codes) ', len(sample.node_opcode[0]))
print('edge_index: (#edges) ', len(sample.edge_index[0]))
print('configs: (#configurations) ', len(sample.config_feat[0]))
print('config_feat: (#config_features) ', len(sample.config_feat[0][0]))
print('config_runtime: (#runtimes == # configs) ', len(sample.config_runtime[0]))
print('config_runtime_normalizers: (#runtimes_normalized == #configs) ', len(sample.config_runtime_normalizers[0]))

no of nodes: (#nodes) in computation graph  31
node_feat: (#features_of_a_node)  140
node_opcode: (#operation_codes)  31
edge_index: (#edges)  33
configs: (#configurations)  1537
config_feat: (#config_features)  24
config_runtime: (#runtimes == # configs)  1537
config_runtime_normalizers: (#runtimes_normalized == #configs)  1537


In [31]:
sample = tilex_train.head(2)
print('no of nodes: (#nodes) in computation graph ', len(sample.node_feat[1]))
print('node_feat: (#features_of_a_node) ', len(sample.node_feat[1][0]))
print('node_opcode: (#operation_codes) ', len(sample.node_opcode[1]))
print('edge_index: (#edges) ', len(sample.edge_index[1]))
print('configs: (#configurations) ', len(sample.config_feat[1]))
print('config_feat: (#config_features) ', len(sample.config_feat[1][0]))
print('config_runtime: (#runtimes == # configs) ', len(sample.config_runtime[1]))
print('config_runtime_normalizers: (#runtimes_normalized == #configs) ', len(sample.config_runtime_normalizers[1]))

no of nodes: (#nodes) in computation graph  25
node_feat: (#features_of_a_node)  140
node_opcode: (#operation_codes)  25
edge_index: (#edges)  28
configs: (#configurations)  8407
config_feat: (#config_features)  24
config_runtime: (#runtimes == # configs)  8407
config_runtime_normalizers: (#runtimes_normalized == #configs)  8407


In [32]:
class gConv(nn.Module):
    def __init__(self, in_feat, out_feat, batch_size):
        super(gConv, self).__init__()
        self.in_feat = in_feat
        self.out_feat = out_feat
        # try nn.init xavier_normal_ or xavier_uniform_ if convergence is slow
        self.w = nn.Parameter(torch.stack([torch.randn(self.in_feat, self.out_feat, device=device)]*batch_size, axis=0), requires_grad=True)
        
    def forward(self, adj_matrix, feats):
        adj_matrix = adj_matrix.to(device)
        feats = feats.to(device)
        id_matrix = torch.eye(feats.shape[1], device=device)
        id_matrix = torch.stack([id_matrix]*feats.shape[0], axis=0)
        adj_matrix = adj_matrix + id_matrix
        adj_matrix = kipf_norm(adj_matrix)
        agg_feat = torch.matmul(torch.matmul(adj_matrix, feats), self.w)
        return torch.relu(agg_feat)

In [33]:
gconv = gConv(in_feat=3, out_feat=2, batch_size=2) # feats in, feats out
gconv = gconv.to(device) # need re-assign
adj_matrix = torch.FloatTensor([[0, 1, 1], 
              [1, 0, 0], 
              [1, 0, 0]])
feats = torch.FloatTensor([[1, 2, 3], 
                           [2, 1, 2], 
                           [3, 1, 5]])
x = gconv(torch.unsqueeze(adj_matrix, 0), torch.unsqueeze(feats, 0))
x

tensor([[[1.0466, 2.0644],
         [0.9927, 1.4407],
         [0.7985, 1.6518]],

        [[1.0466, 2.0644],
         [0.9927, 1.4407],
         [0.7985, 1.6518]]], grad_fn=<ReluBackward0>)

In [34]:
class GNN(nn.Module):
    def __init__(self, input_feat, batch_size):
        super(GNN, self).__init__()
        self.input_feat = input_feat
        self.gConv1 = gConv(input_feat, 256, batch_size)
        self.gConv1.to(device)
        self.gConv2 = gConv(256, 512, batch_size)
        self.gConv2.to(device)
        self.gConv3 = gConv(512, 128, batch_size)
        self.gConv3.to(device)
        self.gConv4 = gConv(128, 64, batch_size)
        self.gConv4.to(device)
        self.gConv5 = gConv(64, 32, batch_size)
        self.gConv5.to(device)
    
    def forward(self, inputs):
        op_code, node_config_feat, adj_matrix, topo_order = inputs
        op_code, node_config_feat, adj_matrix, topo_order = op_code, node_config_feat, adj_matrix, topo_order
        embd_layer = nn.Embedding(120, 10)
        # type casting
        op_code = op_code.clone().detach().to(torch.int64)
        adj_matrix = adj_matrix.clone().detach().to(torch.float)
        topo_order = topo_order.clone().detach().to(torch.int64)
        # type casting
        op_embd = embd_layer(op_code)
        feats = torch.cat((op_embd, node_config_feat), dim=-1)
        # type casting
        feats = feats.clone().detach().to(torch.float)
        # type casting
        assert (feats.shape[-1] == (10+140+24))
        # graph convolutions
        x = self.gConv1(adj_matrix, feats)
        x = self.gConv2(adj_matrix, x)
        x = self.gConv3(adj_matrix, x)
        x = self.gConv4(adj_matrix, x)
        x = self.gConv5(adj_matrix, x)
        assert(x.shape[1] == feats.shape[1])
        assert(x.shape[2] == 32)
        # topological order aware downsampling
        topo_embd_layer = nn.Embedding(feats.shape[1], 10)
        topo_embd = topo_embd_layer(topo_order)
        topo_embd = torch.transpose(topo_embd, 1, 2)
        assert(topo_embd.shape[1] == 10)
        assert(topo_embd.shape[2] == feats.shape[1])
        latent_feat = torch.matmul(topo_embd.to(device), x.to(device))
        flat_feat = torch.flatten(latent_feat, start_dim=1)
        flat_dim = flat_feat.shape[1]
        
        # linear layers init
        self.dense1 = nn.Linear(flat_dim, 256).to(device)
        self.bn1 = nn.BatchNorm1d(256).to(device)
        self.dense2 = nn.Linear(256, 128).to(device)
        self.bn2 = nn.BatchNorm1d(128).to(device)
        self.dense3 = nn.Linear(128, 32).to(device)
        self.bn3 = nn.BatchNorm1d(32).to(device)
        self.dense4 = nn.Linear(32, 8).to(device)
        self.bn4 = nn.BatchNorm1d(8).to(device)
        self.dense5 = nn.Linear(8, 1).to(device)
        self.act = nn.ReLU().to(device)
        # linear layers build
        x = self.act(self.dense1(flat_feat))
        x = self.bn1(x)
        x = self.act(self.dense2(x))
        x = self.bn2(x)
        x = self.act(self.dense3(x))
        x = self.bn3(x)
        x = self.act(self.dense4(x))
        x = self.bn4(x)
        x = self.act(self.dense5(x))
        return x

**TRAINING TILE**

In [35]:
batch_size = 64 # determine the batch_size closely (also frame a algorithmic question related to this and post in on LeetCode)
model = GNN(174, batch_size)
model = model.to(device)

In [36]:
train1_pkl = "/kaggle/input/processed-tile-data-1/tiler_train1.pkl"
train2_pkl = "/kaggle/input/processed-tile-data-2/tiler_train2.pkl"
val_pkl = "/kaggle/input/valid-fast-slow/tiler_val.pkl"
train1 = True
if train1:
    tile_train = pd.read_pickle(train1_pkl)
else:
    tile_train = pd.read_pickle(train2_pkl)
tile_val = pd.read_pickle(val_pkl)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/processed-tile-data-1/tiler_train1.pkl'

In [ ]:
print("train data rows: ", len(tile_train))
# slice the data to make it compatible with the batch_size
train_just = tile_train.iloc[0:5546240].copy(deep=True)
train_just.head()

In [ ]:
print("valid data rows: ", len(tile_val))
# slice the data to make it compatible with the batch_size
val_just = tile_val.iloc[0:1042048].copy(deep=True)
val_just.head()

**TRAINING MODEL**

In [ ]:
test = True
if test:
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    num_epochs = 0 # change the no of epochs to train it
    train_batchs = dataset(train_just)
    val_batchs = dataset(val_just)
    train_batchs = DataLoader(train_batchs, batch_size=batch_size, shuffle=True)
    val_batchs = DataLoader(val_batchs, batch_size=batch_size, shuffle=True) 
    for epoch in range(num_epochs):
        tot_loss = 20
        for batch in train_batchs:
            batch_feats, batch_true = batch
            optimizer.zero_grad()
            batch_preds = model(batch_feats)
            batch_preds = batch_preds.view(batch_size,)
            loss = criterion(batch_preds.to(device), batch_true.to(device))
            tot_loss += loss.item()
            # add high weightage to top 5 fastest runtime config (ranking loss)
            loss.backward()
            optimizer.step()
        print('train tot_loss: ', tot_loss, end='  ')
        tot_loss = 0
        with torch.no_grad():
            for batch in val_batchs:
                batch_feats, batch_true = batch
                batch_preds = model(batch_feats)
                batch_preds = batch_preds.view(batch_size,)
                tot_loss += criterion(batch_preds.to(device), batch_true.to(device)).item()
        print('valid tot_loss: ', tot_loss)

In [ ]:
checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}
model_path = '/kaggle/working/trained_model.pth'
torch.save(checkpoint, model_path)

In [ ]:
load = False
if load:
    model = GNN()
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

In [ ]:
test_data = tile_xla["test"]
print(len(test_data))
test_data.head()

In [ ]:
if False: # Change it to True to run the cell
    model.eval()
    result = []
    for i in range(2): # len(test_data)
        row = test_data.iloc[i].to_frame().T
        row = row.copy(deep=True)
        row['topo_order'] = row['edge_index'].map(lambda e: topo_sort(e))
        row['edge_index'] = row['edge_index'].map(lambda e: adj_mat(e, max([max(edge) for edge in e])+1))
        made_row = make_data(row, i)
        rank = []
        for config in range(2): # len(made_row)
            econ = made_row.iloc[config].to_frame().T
            econ_batch = dataset(econ)
            econ_data = DataLoader(econ_batch, batch_size=1, shuffle=False)
            for batch in econ_data:
                batch_feats, batch_labels = batch
                pred = model(batch_feats)
                pred_mean = torch.mean(pred).item()
                print(pred_mean, batch_labels.item())
            rank.append((config, pred_mean))
        rank.sort(key=lambda x: x[1])
        # take top 5 and append to sample_submissions.csv
        result.append([x[0] for x in rank[:5]])

In [ ]:

if False: # Change it to True to get the inference
    assert(len(result) == 844)
    sub = pd.read_csv("/kaggle/input/predict-ai-model-runtime/sample_submission.csv")
    for i, res in enumerate(result):
        s = ""
        for config in res:
            s += str(config)+';'
        s = s[:-1]
        sub.iloc[i].TopConfigs = s
    sub.to_csv("/kaggle/working/sub.csv", index=False)